# Notebook 05: Denoising en CT - LDCT a NDCT

Este notebook implementa tecnicas de reduccion de ruido (denoising) en imagenes CT pulmonares.

## Contenido:
1. **Dataset Mayo Clinic (TCIA)**: Pares reales NDCT/LDCT (descarga automatica)
2. **Alternativa**: Simulacion de ruido LDCT desde LUNA16
3. **Tecnicas de Denoising**: Filtros clasicos vs Deep Learning (DnCNN)
4. **Metricas de Calidad**: PSNR, SSIM, MSE

---

## Datasets

| Dataset | Tipo | Tamano | Descarga |
|---------|------|--------|----------|
| **Mayo Clinic LDCT** | Pares reales NDCT/LDCT | ~200 GB (Chest) | Automatica via tcia_utils |
| **LUNA16** | Solo NDCT (simular ruido) | ~13 GB | Automatica |

**Flujo del notebook:**
1. Intentar descargar/usar datos reales Mayo Clinic
2. Si no disponible, usar LUNA16 con ruido simulado

---

## 1. Configuración del Entorno

In [ ]:
# Configuracion del entorno
import sys
import os
from pathlib import Path
import pandas as pd

IN_COLAB = 'google.colab' in sys.modules

# ============================================================
# CONFIGURACION
# ============================================================
USE_MAYO_PROJECTIONS = True  # Usar proyecciones/sinogramas Mayo
USE_MAYO_IMAGES = True       # Usar imagenes CT reconstruidas Mayo
MAX_MAYO_PATIENTS = 5        # Pacientes a usar (None = todos)

if IN_COLAB:
    print("Ejecutando en Google Colab")
    print("="*50)

    # Instalar dependencias
    import subprocess
    paquetes = ['SimpleITK', 'scikit-image', 'tcia_utils', 'pydicom']
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + paquetes)

    # Clonar repositorio
    repo_url = "https://github.com/Daspony/Imagenes-Biomedicas.git"
    if not os.path.exists("/content/Imagenes-Biomedicas"):
        subprocess.run(["git", "clone", repo_url], cwd="/content", check=True)
    sys.path.insert(0, "/content/Imagenes-Biomedicas")

    project_root = "/content/Imagenes-Biomedicas"

    # Montar Drive para guardar datos y modelos
    from google.colab import drive
    drive.mount('/content/drive')

    # Rutas en Drive (persistentes)
    drive_data = "/content/drive/MyDrive/CT_Denoising"
    os.makedirs(drive_data, exist_ok=True)

    # Rutas Mayo separadas
    MAYO_PROJ_PATH = os.path.join(drive_data, "Mayo_LDCT")    # Proyecciones/sinogramas
    MAYO_IMG_PATH = os.path.join(drive_data, "Mayo_Images")   # Imagenes CT reconstruidas
    weights_dir = os.path.join(drive_data, "weights")

    # LUNA16 para fallback
    LUNA16_PATH = os.path.join(project_root, "LUNA16")

    print(f"[OK] Datos se guardaran en: {drive_data}")
else:
    print("Ejecutando localmente")
    print("="*50)

    # Instalar tcia_utils si no esta
    try:
        import tcia_utils
    except ImportError:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "tcia_utils", "pydicom"])

    parent_dir = os.path.abspath('..')
    if parent_dir not in sys.path:
        sys.path.insert(0, parent_dir)

    project_root = parent_dir

    # Rutas Mayo separadas (local)
    MAYO_PROJ_PATH = os.path.join(project_root, "Mayo_LDCT")
    MAYO_IMG_PATH = os.path.join(project_root, "Mayo_Images")

    LUNA16_PATH = os.path.join(project_root, "LUNA16")
    weights_dir = os.path.join(project_root, "weights")

# Crear directorios
os.makedirs(weights_dir, exist_ok=True)

# Verificar disponibilidad de cada dataset
MAYO_PROJ_AVAILABLE = os.path.exists(MAYO_PROJ_PATH) and any(Path(MAYO_PROJ_PATH).rglob("*.dcm"))
MAYO_IMG_AVAILABLE = os.path.exists(MAYO_IMG_PATH) and any(Path(MAYO_IMG_PATH).rglob("*.dcm"))

# Para compatibilidad, usar ruido simulado si no hay datos reales
USE_SIMULATED_NOISE = not (MAYO_PROJ_AVAILABLE or MAYO_IMG_AVAILABLE)

print(f"\n[INFO] Estado de datos:")
print(f"  - Proyecciones Mayo: {'Disponible' if MAYO_PROJ_AVAILABLE else 'No disponible'}")
print(f"  - Imagenes CT Mayo: {'Disponible' if MAYO_IMG_AVAILABLE else 'No disponible'}")
print(f"  - Ruido simulado: {'Activo' if USE_SIMULATED_NOISE else 'No necesario'}")


In [ ]:
# ============================================================
# DESCARGA AUTOMATICA - MAYO CLINIC LDCT (TCIA)
# ============================================================
from tcia_utils import nbia
from tqdm import tqdm

def download_mayo_ldct(download_path, body_part="CHEST", max_patients=5):
    """
    Descarga pares NDCT/LDCT del dataset Mayo Clinic (TCIA)
    Con barra de progreso para monitorear la descarga.
    """
    os.makedirs(download_path, exist_ok=True)
    
    # Verificar si ya hay datos
    existing_files = list(Path(download_path).rglob("*.dcm"))
    if len(existing_files) > 100:
        print(f"[OK] Dataset Mayo ya existe: {len(existing_files)} archivos DICOM")
        return True
    
    print(f"[INFO] Buscando series de {body_part} en TCIA...")
    
    try:
        # Obtener series del dataset Mayo LDCT
        series = nbia.getSeries(collection="LDCT-and-Projection-data")
        
        if series is None or len(series) == 0:
            print("[WARNING] No se encontraron series en TCIA")
            return False
        
        # Convertir a DataFrame para filtrar
        df = pd.DataFrame(series)
        print(f"[INFO] Series totales: {len(df)}")
        
        # Filtrar por body part
        if 'BodyPartExamined' in df.columns:
            df_filtered = df[df['BodyPartExamined'].str.upper() == body_part.upper()]
            print(f"[INFO] Series de {body_part}: {len(df_filtered)}")
        else:
            df_filtered = df
        
        if len(df_filtered) == 0:
            print(f"[WARNING] No hay series de {body_part}")
            return False
        
        # Limitar numero de pacientes
        if max_patients and 'PatientID' in df_filtered.columns:
            unique_patients = df_filtered['PatientID'].unique()[:max_patients]
            df_filtered = df_filtered[df_filtered['PatientID'].isin(unique_patients)]
            print(f"[INFO] Limitando a {len(unique_patients)} pacientes ({len(df_filtered)} series)")
        
        # Descargar serie por serie con progreso
        print(f"\n[INFO] Descargando {len(df_filtered)} series...")
        series_list = df_filtered.to_dict('records')
        
        successful = 0
        failed = 0
        
        for i, serie in enumerate(tqdm(series_list, desc="Descargando series")):
            try:
                # Descargar una serie a la vez
                nbia.downloadSeries(
                    [serie],  # Lista con una sola serie
                    path=download_path,
                    csv_filename=None  # No crear CSV por cada serie
                )
                successful += 1
            except Exception as e:
                failed += 1
                print(f"\n[WARNING] Error en serie {i+1}: {e}")
        
        print(f"\n[OK] Descarga completada: {successful} exitosas, {failed} fallidas")
        print(f"[OK] Datos en: {download_path}")
        return successful > 0
        
    except Exception as e:
        print(f"[ERROR] Error descargando Mayo: {e}")
        return False


# ============================================================
# INTENTAR OBTENER DATOS MAYO
# ============================================================
print("="*60)
print("DATASET MAYO CLINIC LDCT")
print("="*60)

MAYO_AVAILABLE = False

if USE_MAYO_DATA:
    # Verificar si ya existe
    existing_dcm = list(Path(MAYO_PATH).rglob("*.dcm")) if os.path.exists(MAYO_PATH) else []
    
    if len(existing_dcm) > 100:
        MAYO_AVAILABLE = True
        print(f"[OK] Mayo LDCT encontrado: {len(existing_dcm)} archivos DICOM")
    else:
        print("[INFO] Intentando descargar Mayo LDCT...")
        MAYO_AVAILABLE = download_mayo_ldct(
            download_path=MAYO_PATH,
            body_part="CHEST",
            max_patients=MAX_MAYO_PATIENTS
        )
else:
    print("[INFO] USE_MAYO_DATA=False, usando ruido simulado")

print(f"\nMayo LDCT disponible: {MAYO_AVAILABLE}")

In [ ]:
# Importar librerias
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pydicom

# Procesamiento de imagenes
from skimage.restoration import denoise_tv_chambolle, denoise_bilateral, denoise_nl_means
from skimage.filters import gaussian
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from scipy.ndimage import median_filter

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Importar modulos del proyecto
from utils import LUNA16DataLoader, download_luna16

# Configurar dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# ============================================================
# CONFIGURAR FUENTE DE DATOS
# ============================================================
USE_SIMULATED_NOISE = not MAYO_AVAILABLE

if MAYO_AVAILABLE:
    print("\n[OK] Usando datos REALES de Mayo Clinic (NDCT/LDCT)")
else:
    print("\n[INFO] Mayo no disponible - Usando LUNA16 con ruido SIMULADO")
    
    # Configurar LUNA16
    DATA_PATH = os.path.join(LUNA16_PATH, 'subset0')
    ANNOTATIONS_PATH = os.path.join(LUNA16_PATH, 'annotations.csv')
    
    # Descargar si no existe
    if not os.path.exists(DATA_PATH):
        print("[INFO] Descargando LUNA16 subset0...")
        download_luna16(subsets=[0], download_dir=LUNA16_PATH)
    
    mhd_count = len(list(Path(DATA_PATH).glob("*.mhd"))) if os.path.exists(DATA_PATH) else 0
    print(f"  LUNA16 path: {DATA_PATH}")
    print(f"  Scans disponibles: {mhd_count}")

print("\nLibrerias importadas")

In [ ]:
# ============================================================
# CARGAR DATOS DE EJEMPLO
# ============================================================

if USE_SIMULATED_NOISE:
    # Cargar un escaneo de LUNA16 para demo
    loader = LUNA16DataLoader(DATA_PATH, ANNOTATIONS_PATH)
    mhd_files = list(Path(DATA_PATH).glob("*.mhd"))
    print(f"Escaneos LUNA16 disponibles: {len(mhd_files)}")
    
    # Cargar escaneo
    sample_path = str(mhd_files[0])
    ct_scan, origin, spacing = loader.load_itk_image(sample_path)
    print(f"Volumen cargado: {ct_scan.shape}")
    print(f"Spacing: {spacing}")
    
    # Seleccionar un slice central
    slice_idx = ct_scan.shape[0] // 2
    original_slice = ct_scan[slice_idx].copy()
    print(f"Slice seleccionado: {slice_idx}")
    print(f"Rango HU: [{original_slice.min():.0f}, {original_slice.max():.0f}]")
else:
    # Cargar un escaneo de Mayo para demo (se hara en celdas posteriores)
    print("[INFO] Datos Mayo se cargaran en seccion dedicada")
    original_slice = None

---

# Modelo DnCNN y Funciones Comunes

Estas definiciones son compartidas por ambas partes (ruido simulado y datos reales).

---

---

## Arquitectura DnCNN

**DnCNN** (Denoising Convolutional Neural Network) es una arquitectura clásica para denoising:

- Aprende el **residuo** (ruido) en lugar de la imagen limpia
- Usa batch normalization y ReLU
- Típicamente 17-20 capas convolucionales

$$\hat{x} = y - f(y; \theta)$$

donde $y$ es la imagen ruidosa, $f$ predice el ruido, y $\hat{x}$ es la imagen denoised.

In [ ]:
class DnCNN(nn.Module):
    """
    DnCNN para denoising de imágenes CT
    
    Arquitectura:
    - Conv + ReLU (primera capa)
    - (Conv + BN + ReLU) x (depth-2) capas intermedias
    - Conv (última capa)
    
    El modelo predice el residuo (ruido) que se resta de la entrada.
    """
    def __init__(self, in_channels=1, out_channels=1, num_features=64, depth=17):
        super(DnCNN, self).__init__()
        
        layers = []
        
        # Primera capa: Conv + ReLU
        layers.append(nn.Conv2d(in_channels, num_features, kernel_size=3, padding=1, bias=False))
        layers.append(nn.ReLU(inplace=True))
        
        # Capas intermedias: Conv + BN + ReLU
        for _ in range(depth - 2):
            layers.append(nn.Conv2d(num_features, num_features, kernel_size=3, padding=1, bias=False))
            layers.append(nn.BatchNorm2d(num_features))
            layers.append(nn.ReLU(inplace=True))
        
        # Última capa: Conv (sin activación)
        layers.append(nn.Conv2d(num_features, out_channels, kernel_size=3, padding=1, bias=False))
        
        self.dncnn = nn.Sequential(*layers)
    
    def forward(self, x):
        """Predice el residuo y lo resta de la entrada"""
        residual = self.dncnn(x)
        return x - residual  # Imagen denoised


# Crear modelo
model_dncnn = DnCNN(in_channels=1, out_channels=1, num_features=64, depth=17).to(device)

print("DnCNN creado")
print(f"Parámetros totales: {sum(p.numel() for p in model_dncnn.parameters()):,}")
print(f"Dispositivo: {device}")

---

## Funcion de Entrenamiento

In [ ]:
def train_denoiser(model, dataloader, epochs=10, lr=1e-3):
    """
    Entrena el modelo de denoising
    
    Args:
        model: Modelo DnCNN
        dataloader: DataLoader con pares (ruidosa, limpia)
        epochs: Número de épocas
        lr: Learning rate
    
    Returns:
        history: Diccionario con historial de entrenamiento
    """
    model.train()
    
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    
    history = {'loss': [], 'psnr': []}
    
    for epoch in range(epochs):
        epoch_loss = 0
        epoch_psnr = 0
        num_batches = 0
        
        pbar = tqdm(dataloader, desc=f'Época {epoch+1}/{epochs}')
        
        for noisy, clean in pbar:
            noisy = noisy.to(device)
            clean = clean.to(device)
            
            # Forward
            optimizer.zero_grad()
            denoised = model(noisy)
            loss = criterion(denoised, clean)
            
            # Backward
            loss.backward()
            optimizer.step()
            
            # Métricas
            epoch_loss += loss.item()
            
            # Calcular PSNR
            with torch.no_grad():
                mse = F.mse_loss(denoised, clean)
                batch_psnr = 10 * torch.log10(1.0 / mse)
                epoch_psnr += batch_psnr.item()
            
            num_batches += 1
            pbar.set_postfix({'Loss': f'{loss.item():.6f}', 'PSNR': f'{batch_psnr.item():.2f}'})
        
        scheduler.step()
        
        avg_loss = epoch_loss / num_batches
        avg_psnr = epoch_psnr / num_batches
        
        history['loss'].append(avg_loss)
        history['psnr'].append(avg_psnr)
        
        print(f'Época {epoch+1}: Loss = {avg_loss:.6f}, PSNR = {avg_psnr:.2f} dB')
    
    return history


print("Función de entrenamiento definida")

---

## Funcion de Evaluacion

In [ ]:
def evaluate_denoiser(model, image_clean_hu, dose_ratio=0.25, I0=1e5):
    """
    Evalua el modelo de denoising en una imagen.
    Usa simulacion de ruido en dominio del sinograma (Yu et al. 2012).
    
    Args:
        model: Modelo entrenado
        image_clean_hu: Imagen limpia en valores HU
        dose_ratio: Ratio de dosis para simular LDCT
        I0: Fotones incidentes
    
    Returns:
        dict: Resultados con imagenes y metricas
    """
    model.eval()
    
    # Simular LDCT en dominio del sinograma
    noisy_hu = add_ldct_noise_yu2012(image_clean_hu, dose_ratio=dose_ratio, I0=I0)
    
    # Normalizar para el modelo
    def normalize(img):
        img_clipped = np.clip(img, -1000, 400)
        return (img_clipped - (-1000)) / (400 - (-1000))
    
    clean = normalize(image_clean_hu)
    noisy = normalize(noisy_hu)
    
    # Preparar tensor
    noisy_tensor = torch.FloatTensor(noisy).unsqueeze(0).unsqueeze(0).to(device)
    
    # Inferencia
    with torch.no_grad():
        denoised_tensor = model(noisy_tensor)
    
    denoised = denoised_tensor.squeeze().cpu().numpy()
    denoised = np.clip(denoised, 0, 1)
    
    # Calcular metricas
    psnr_noisy = psnr(clean, noisy, data_range=1.0)
    psnr_denoised = psnr(clean, denoised, data_range=1.0)
    
    ssim_noisy = ssim(clean, noisy, data_range=1.0)
    ssim_denoised = ssim(clean, denoised, data_range=1.0)
    
    return {
        'clean': clean,
        'noisy': noisy,
        'denoised': denoised,
        'psnr_noisy': psnr_noisy,
        'psnr_denoised': psnr_denoised,
        'ssim_noisy': ssim_noisy,
        'ssim_denoised': ssim_denoised,
        'psnr_gain': psnr_denoised - psnr_noisy,
        'ssim_gain': ssim_denoised - ssim_noisy
    }


print("Funcion de evaluacion definida (usa ruido en sinograma)")

---

# PARTE A: Ruido Simulado 
Esta seccion permite trabajar con **ruido sintetico** simulado en el dominio del sinograma.
Util para desarrollo y pruebas sin necesidad de descargar datasets externos.

**Contenido:**
- Modelo de ruido CT (Yu et al. 2012)
- Variacion de niveles de dosis
- Tecnicas clasicas de denoising
- DnCNN con ruido simulado

---

---

## 2. Modelo de Ruido en CT (Yu et al. 2012)

El ruido en CT se origina en las **proyecciones** (sinograma), no en la imagen reconstruida.

### Modelo Fisico

Para un rayo de rayos X que atraviesa el objeto:

$$P = -\ln\left(\frac{N}{N_0}\right) = \int \mu(x) \, dx$$

Donde:
- $N_0$ = fotones incidentes
- $N$ = fotones detectados (sigue distribucion Poisson)
- $P$ = proyeccion (line integral de atenuacion)
- $\mu$ = coeficiente de atenuacion

### Simulacion de Dosis Reducida (Eq. 6 del paper)

Para simular una imagen con dosis reducida $a$ (ej: $a=0.25$ para 25%):

$$\tilde{P}_B \approx P_A + \sqrt{\frac{1-a}{a} \cdot \frac{\exp(P_A)}{N_{0A}}} \cdot x$$

Donde $x \sim N(0,1)$ es ruido gaussiano.

### Con Ruido Electronico (Eq. 11)

$$\tilde{P}_B \approx P_A + \sqrt{\frac{1-a}{a} \cdot \frac{\exp(P_A)}{N_{0A}} \cdot \left(1 + \frac{1+a}{a} \cdot \frac{N_e \cdot \exp(P_A)}{N_{0A}}\right)} \cdot x$$

### Pipeline de Simulacion

```
Imagen NDCT -> Sinograma (Radon) -> +Ruido (Eq.6) -> Reconstruccion (FBP) -> Imagen LDCT
```

**Referencia**: Yu L, et al. "Development and Validation of a Practical Lower-Dose-Simulation Tool for Optimizing CT Scan Protocols". J Comput Assist Tomogr 2012;36:477-487.

In [ ]:
# ============================================================
# SIMULACION DE RUIDO LDCT - FIEL AL PAPER YU ET AL. 2012
# ============================================================
# Referencia: Yu et al. "Development and Validation of a Practical 
# Lower-Dose-Simulation Tool for Optimizing CT Scan Protocols"
# J Comput Assist Tomogr 2012;36:477-487
#
# El ruido se simula en el dominio del SINOGRAMA, no en la imagen.
# ============================================================

from skimage.transform import radon, iradon

def image_to_sinogram(image, theta=None):
    """
    Convierte imagen CT a sinograma usando transformada de Radon.
    
    Args:
        image: Imagen CT 2D (valores HU o normalizados)
        theta: Angulos de proyeccion (default: 0-180 grados)
    
    Returns:
        sinogram: Sinograma (proyecciones)
        theta: Angulos utilizados
    """
    if theta is None:
        theta = np.linspace(0., 180., max(image.shape), endpoint=False)
    
    sinogram = radon(image, theta=theta, circle=True)
    return sinogram, theta


def sinogram_to_image(sinogram, theta, output_size=None):
    """
    Reconstruye imagen desde sinograma usando retroproyeccion filtrada.
    
    Args:
        sinogram: Sinograma
        theta: Angulos de proyeccion
        output_size: Tamano de salida (default: automatico)
    
    Returns:
        image: Imagen reconstruida
    """
    image = iradon(sinogram, theta=theta, output_size=output_size, circle=True)
    return image


def add_noise_sinogram_domain(image, dose_ratio=0.25, I0=1e5, include_electronic=True, Ne=8.2):
    """
    Simula LDCT anadiendo ruido en el dominio del sinograma.
    Implementacion fiel al paper Yu et al. 2012 (Eq. 6 y 11).
    
    El modelo de ruido es:
        P_B = P_A + sqrt((1-a)/a * exp(P_A)/N0_A) * x      (Eq. 6)
    
    Con ruido electronico (Eq. 11):
        P_B = P_A + sqrt((1-a)/a * exp(P_A)/N0_A * (1 + (1+a)/a * Ne*exp(P_A)/N0_A)) * x
    
    Args:
        image: Imagen NDCT original (valores HU)
        dose_ratio: Ratio de dosis simulada (a). Ej: 0.25 = 25% de dosis original
        I0: Numero de fotones incidentes (N0) - parametro del escaner
        include_electronic: Incluir ruido electronico
        Ne: Ruido electronico equivalente (calibrado en paper como ~8.2)
    
    Returns:
        noisy_image: Imagen LDCT simulada
        sinogram_noisy: Sinograma con ruido (para visualizacion)
        sinogram_clean: Sinograma original (para visualizacion)
    """
    # 1. Normalizar imagen a coeficientes de atenuacion positivos
    #    HU = 1000 * (mu - mu_water) / mu_water
    #    => mu/mu_water = HU/1000 + 1
    #    Usamos valores positivos para el sinograma
    mu_water = 0.02  # cm^-1 aproximado para agua a ~70 keV
    
    # Convertir HU a coeficientes de atenuacion relativos (positivos)
    # Asumimos que la imagen esta en HU
    image_hu = image.copy()
    
    # Escalar a valores de atenuacion (0 = aire, 1 = agua, >1 = hueso)
    image_atten = (image_hu / 1000.0) + 1.0
    image_atten = np.clip(image_atten, 0.001, None)  # Evitar valores negativos/cero
    
    # 2. Calcular sinograma (transformada de Radon)
    #    El sinograma representa la integral de linea de atenuacion
    sinogram_clean, theta = image_to_sinogram(image_atten)
    
    # 3. Convertir a proyecciones P = -ln(N/N0) = integral de mu
    #    En realidad, radon ya nos da la integral, que es proporcional a P
    #    P = sum(mu * dx) a lo largo del rayo
    P_A = sinogram_clean.copy()
    
    # Escalar para que los valores sean realistas
    # (el sinograma de radon no tiene unidades fisicas exactas)
    P_A = P_A * 0.01  # Factor de escala empirico
    
    # 4. Calcular N0_A (fotones incidentes) para cada detector
    #    Simplificacion: asumimos N0 uniforme (sin bowtie filter)
    #    En un escaner real, esto varia con el detector (Fig. 3 del paper)
    N0_A = I0
    
    # 5. Aplicar modelo de ruido (Eq. 6 del paper)
    #    P_B = P_A + sqrt((1-a)/a * exp(P_A)/N0_A) * x
    a = dose_ratio
    
    # Varianza del ruido segun Eq. 6
    # var = (1-a)/a * exp(P_A) / N0_A
    exp_P = np.exp(P_A)
    variance_base = ((1.0 - a) / a) * (exp_P / N0_A)
    
    # 6. Incluir ruido electronico si se especifica (Eq. 11)
    if include_electronic:
        # var = (1-a)/a * exp(P)/N0 * (1 + (1+a)/a * Ne*exp(P)/N0)
        electronic_term = 1.0 + ((1.0 + a) / a) * (Ne * exp_P / N0_A)
        variance = variance_base * electronic_term
    else:
        variance = variance_base
    
    # Asegurar varianza no negativa
    variance = np.clip(variance, 0, None)
    
    # 7. Generar ruido gaussiano y aplicar
    x = np.random.randn(*P_A.shape)
    noise = np.sqrt(variance) * x
    
    P_B = P_A + noise
    
    # 8. Convertir de vuelta a sinograma para reconstruccion
    sinogram_noisy = P_B / 0.01  # Deshacer factor de escala
    
    # 9. Reconstruir imagen desde sinograma ruidoso
    image_recon_atten = sinogram_to_image(sinogram_noisy, theta, output_size=image.shape[0])
    
    # 10. Convertir de atenuacion a HU
    noisy_image = (image_recon_atten - 1.0) * 1000.0
    
    return noisy_image, sinogram_noisy, sinogram_clean


def add_ldct_noise_yu2012(image, dose_ratio=0.25, I0=1e5):
    """
    Wrapper simple para simular LDCT segun Yu et al. 2012.
    
    Args:
        image: Imagen NDCT en valores HU
        dose_ratio: Fraccion de dosis (0.25 = 25% = quarter-dose)
        I0: Fotones incidentes (ajustar segun nivel de ruido deseado)
    
    Returns:
        noisy_image: Imagen LDCT simulada
    """
    noisy_image, _, _ = add_noise_sinogram_domain(
        image, 
        dose_ratio=dose_ratio, 
        I0=I0,
        include_electronic=True,
        Ne=8.2
    )
    return noisy_image.astype(image.dtype)


print("Funciones de ruido implementadas (Yu et al. 2012):")
print("  - image_to_sinogram(): Imagen -> Sinograma (Radon)")
print("  - sinogram_to_image(): Sinograma -> Imagen (FBP)")
print("  - add_noise_sinogram_domain(): Ruido en sinograma (Eq. 6, 11)")
print("  - add_ldct_noise_yu2012(): Wrapper simple para LDCT")
print("\nModelo: P_B = P_A + sqrt((1-a)/a * exp(P_A)/N0) * x")

In [ ]:
# Aplicar simulacion LDCT en dominio del sinograma (Yu et al. 2012)
if USE_SIMULATED_NOISE and original_slice is not None:
    print("Simulando LDCT en dominio del sinograma...")
    print("Proceso: Imagen -> Sinograma -> +Ruido -> Reconstruccion")
    
    # Simular diferentes niveles de dosis
    ldct_25, sino_noisy_25, sino_clean = add_noise_sinogram_domain(
        original_slice, dose_ratio=0.25, I0=1e5
    )
    ldct_50, sino_noisy_50, _ = add_noise_sinogram_domain(
        original_slice, dose_ratio=0.50, I0=1e5
    )
    ldct_10, sino_noisy_10, _ = add_noise_sinogram_domain(
        original_slice, dose_ratio=0.10, I0=1e5
    )
    
    # Normalizar para visualizacion
    def normalize_for_display(img, min_hu=-1000, max_hu=400):
        img_clipped = np.clip(img, min_hu, max_hu)
        return (img_clipped - min_hu) / (max_hu - min_hu)
    
    orig_display = normalize_for_display(original_slice)
    ldct_25_display = normalize_for_display(ldct_25)
    ldct_50_display = normalize_for_display(ldct_50)
    ldct_10_display = normalize_for_display(ldct_10)
    
    # Para compatibilidad con celdas posteriores
    noisy_ldct = ldct_25
    ldct_display = ldct_25_display
    
    print("[OK] Simulacion LDCT completada")
    print(f"  - 50% dosis: PSNR = {psnr(orig_display, ldct_50_display, data_range=1.0):.2f} dB")
    print(f"  - 25% dosis: PSNR = {psnr(orig_display, ldct_25_display, data_range=1.0):.2f} dB")
    print(f"  - 10% dosis: PSNR = {psnr(orig_display, ldct_10_display, data_range=1.0):.2f} dB")
else:
    print("[INFO] Usando datos reales Mayo - no se simula ruido")
    
    def normalize_for_display(img, min_hu=-1000, max_hu=400):
        img_clipped = np.clip(img, min_hu, max_hu)
        return (img_clipped - min_hu) / (max_hu - min_hu)

In [ ]:
# Visualizar el proceso de simulacion LDCT en dominio del sinograma
if USE_SIMULATED_NOISE and original_slice is not None:
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    
    # Fila 1: Proceso de simulacion
    axes[0, 0].imshow(orig_display, cmap='bone')
    axes[0, 0].set_title('1. Imagen Original\n(NDCT)', fontsize=12)
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(sino_clean, cmap='bone', aspect='auto')
    axes[0, 1].set_title('2. Sinograma\n(Radon Transform)', fontsize=12)
    axes[0, 1].set_xlabel('Angulo')
    axes[0, 1].set_ylabel('Detector')
    
    # Diferencia de sinogramas (ruido anadido)
    sino_diff = sino_noisy_25 - sino_clean
    axes[0, 2].imshow(sino_diff, cmap='RdBu', aspect='auto', 
                      vmin=-np.percentile(np.abs(sino_diff), 99),
                      vmax=np.percentile(np.abs(sino_diff), 99))
    axes[0, 2].set_title('3. Ruido Anadido\n(Eq. 6 Yu et al.)', fontsize=12)
    axes[0, 2].set_xlabel('Angulo')
    axes[0, 2].set_ylabel('Detector')
    
    axes[0, 3].imshow(ldct_25_display, cmap='bone')
    axes[0, 3].set_title('4. LDCT Reconstruido\n(25% dosis)', fontsize=12)
    axes[0, 3].axis('off')
    
    # Fila 2: Comparacion de niveles de dosis
    axes[1, 0].imshow(orig_display, cmap='bone')
    axes[1, 0].set_title('Original (100% dosis)', fontsize=12)
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(ldct_50_display, cmap='bone')
    psnr_50 = psnr(orig_display, ldct_50_display, data_range=1.0)
    axes[1, 1].set_title(f'50% dosis\nPSNR: {psnr_50:.1f} dB', fontsize=12)
    axes[1, 1].axis('off')
    
    axes[1, 2].imshow(ldct_25_display, cmap='bone')
    psnr_25 = psnr(orig_display, ldct_25_display, data_range=1.0)
    axes[1, 2].set_title(f'25% dosis\nPSNR: {psnr_25:.1f} dB', fontsize=12)
    axes[1, 2].axis('off')
    
    axes[1, 3].imshow(ldct_10_display, cmap='bone')
    psnr_10 = psnr(orig_display, ldct_10_display, data_range=1.0)
    axes[1, 3].set_title(f'10% dosis\nPSNR: {psnr_10:.1f} dB', fontsize=12)
    axes[1, 3].axis('off')
    
    plt.suptitle('Simulacion LDCT en Dominio del Sinograma (Yu et al. 2012)', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Visualizar zoom
    fig2, axes2 = plt.subplots(1, 4, figsize=(16, 4))
    y1, y2, x1, x2 = 200, 300, 200, 300
    
    axes2[0].imshow(orig_display[y1:y2, x1:x2], cmap='bone')
    axes2[0].set_title('Original', fontsize=12)
    axes2[0].axis('off')
    
    axes2[1].imshow(ldct_50_display[y1:y2, x1:x2], cmap='bone')
    axes2[1].set_title('50% dosis', fontsize=12)
    axes2[1].axis('off')
    
    axes2[2].imshow(ldct_25_display[y1:y2, x1:x2], cmap='bone')
    axes2[2].set_title('25% dosis', fontsize=12)
    axes2[2].axis('off')
    
    axes2[3].imshow(ldct_10_display[y1:y2, x1:x2], cmap='bone')
    axes2[3].set_title('10% dosis', fontsize=12)
    axes2[3].axis('off')
    
    plt.suptitle('Zoom - Detalle del Ruido', fontsize=14)
    plt.tight_layout()
    plt.show()
    
else:
    print("[SKIP] Visualizacion de ruido simulado - usando datos reales Mayo")

In [ ]:
# Calcular metricas de calidad (solo ruido simulado)
if USE_SIMULATED_NOISE and original_slice is not None:
    print("Metricas de Calidad de Imagen (respecto al original):")
    print("="*60)
    
    metrics = []
    for name, noisy in [('Gaussiano', gauss_display), 
                        ('Poisson', poisson_display), 
                        ('LDCT Sim.', ldct_display)]:
        psnr_val = psnr(orig_display, noisy, data_range=1.0)
        ssim_val = ssim(orig_display, noisy, data_range=1.0)
        mse_val = np.mean((orig_display - noisy)**2)
        
        metrics.append({
            'Tipo': name,
            'PSNR (dB)': f'{psnr_val:.2f}',
            'SSIM': f'{ssim_val:.4f}',
            'MSE': f'{mse_val:.6f}'
        })
        
        print(f"\n{name}:")
        print(f"  PSNR: {psnr_val:.2f} dB")
        print(f"  SSIM: {ssim_val:.4f}")
        print(f"  MSE:  {mse_val:.6f}")
    
    print("\n" + "="*60)
    metrics_df = pd.DataFrame(metrics)
    print(metrics_df.to_string(index=False))
else:
    print("[SKIP] Metricas de ruido simulado - usando datos reales Mayo")

---

## 3. Variación del Nivel de Ruido

Exploramos cómo diferentes niveles de dosis afectan la calidad de imagen.

In [ ]:
# Simular diferentes niveles de dosis (solo con ruido simulado)
if USE_SIMULATED_NOISE and original_slice is not None:
    dose_ratios = [1.0, 0.5, 0.25, 0.125, 0.0625]  # 100%, 50%, 25%, 12.5%, 6.25%
    dose_labels = ['100%', '50%', '25%', '12.5%', '6.25%']

    fig, axes = plt.subplots(2, 5, figsize=(20, 8))

    for i, (dose, label) in enumerate(zip(dose_ratios, dose_labels)):
        if dose == 1.0:
            noisy = original_slice.copy()
        else:
            noisy = add_ct_realistic_noise(original_slice, dose_ratio=dose)
        
        noisy_display = normalize_for_display(noisy)
        
        # Imagen completa
        axes[0, i].imshow(noisy_display, cmap='bone')
        axes[0, i].set_title(f'Dosis: {label}', fontsize=12)
        axes[0, i].axis('off')
        
        # Zoom
        axes[1, i].imshow(noisy_display[200:300, 200:300], cmap='bone')
        
        # Calcular PSNR solo para imagenes con ruido
        if dose < 1.0:
            psnr_val = psnr(orig_display, noisy_display, data_range=1.0)
            axes[1, i].set_title(f'PSNR: {psnr_val:.1f} dB', fontsize=11)
        else:
            axes[1, i].set_title('Referencia', fontsize=11)
        axes[1, i].axis('off')

    plt.suptitle('Efecto del Nivel de Dosis en la Calidad de Imagen CT', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("[SKIP] Visualizacion de niveles de dosis - usando datos reales Mayo")

---

## 4. Técnicas Clásicas de Denoising

Antes de deep learning, se usaban filtros clásicos:

| Técnica | Descripción | Pros | Contras |
|---------|-------------|------|----------|
| Gaussiano | Suavizado isotrópico | Rápido | Pierde bordes |
| Mediana | Filtro no lineal | Preserva bordes | Pierde detalles finos |
| Bilateral | Preserva bordes | Buenos resultados | Lento |
| TV (Total Variation) | Minimiza variación | Preserva bordes | Efecto "cartoon" |
| NLM (Non-Local Means) | Usa parches similares | Muy buenos resultados | Muy lento |

In [ ]:
def apply_classical_denoising(image_normalized):
    """
    Aplica diferentes técnicas clásicas de denoising
    
    Args:
        image_normalized: Imagen normalizada [0, 1]
    
    Returns:
        dict: Diccionario con resultados de cada técnica
    """
    results = {}
    
    # 1. Filtro Gaussiano
    results['Gaussiano'] = gaussian(image_normalized, sigma=1.5)
    
    # 2. Filtro Mediana
    results['Mediana'] = median_filter(image_normalized, size=3)
    
    # 3. Filtro Bilateral
    results['Bilateral'] = denoise_bilateral(
        image_normalized, 
        sigma_color=0.1, 
        sigma_spatial=5,
        channel_axis=None
    )
    
    # 4. Total Variation
    results['TV'] = denoise_tv_chambolle(image_normalized, weight=0.1)
    
    # 5. Non-Local Means (versión rápida)
    results['NLM'] = denoise_nl_means(
        image_normalized,
        h=0.1,
        patch_size=5,
        patch_distance=6,
        channel_axis=None
    )
    
    return results


print("Funciones de denoising clásico definidas")

In [ ]:
# Aplicar denoising a imagen LDCT simulada (solo con ruido simulado)
if USE_SIMULATED_NOISE and original_slice is not None:
    print("Aplicando tecnicas de denoising...")
    print("(Esto puede tomar unos segundos)")

    denoised_results = apply_classical_denoising(ldct_display)

    print("\n[OK] Denoising completado!")
else:
    denoised_results = None
    print("[SKIP] Denoising clasico - usando datos reales Mayo")

In [ ]:
# Visualizar resultados de denoising (solo con ruido simulado)
if USE_SIMULATED_NOISE and denoised_results is not None:
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))

    # Fila 1: Imagenes completas
    axes[0, 0].imshow(orig_display, cmap='bone')
    axes[0, 0].set_title('Original (NDCT)', fontsize=12)
    axes[0, 0].axis('off')

    axes[0, 1].imshow(ldct_display, cmap='bone')
    psnr_noisy = psnr(orig_display, ldct_display, data_range=1.0)
    axes[0, 1].set_title(f'LDCT Simulado\nPSNR: {psnr_noisy:.1f} dB', fontsize=12)
    axes[0, 1].axis('off')

    # Mejores tecnicas
    best_techniques = ['Bilateral', 'NLM']
    for i, tech in enumerate(best_techniques):
        denoised = denoised_results[tech]
        psnr_val = psnr(orig_display, denoised, data_range=1.0)
        axes[0, i+2].imshow(denoised, cmap='bone')
        axes[0, i+2].set_title(f'{tech}\nPSNR: {psnr_val:.1f} dB', fontsize=12)
        axes[0, i+2].axis('off')

    # Fila 2: Zoom
    y1, y2, x1, x2 = 200, 300, 200, 300

    axes[1, 0].imshow(orig_display[y1:y2, x1:x2], cmap='bone')
    axes[1, 0].set_title('Zoom Original', fontsize=12)
    axes[1, 0].axis('off')

    axes[1, 1].imshow(ldct_display[y1:y2, x1:x2], cmap='bone')
    axes[1, 1].set_title('Zoom LDCT', fontsize=12)
    axes[1, 1].axis('off')

    for i, tech in enumerate(best_techniques):
        denoised = denoised_results[tech]
        axes[1, i+2].imshow(denoised[y1:y2, x1:x2], cmap='bone')
        axes[1, i+2].set_title(f'Zoom {tech}', fontsize=12)
        axes[1, i+2].axis('off')

    plt.suptitle('Comparacion de Tecnicas de Denoising', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("[SKIP] Visualizacion de denoising clasico - usando datos reales Mayo")

In [ ]:
# Tabla comparativa de todas las tecnicas (solo con ruido simulado)
if USE_SIMULATED_NOISE and denoised_results is not None:
    print("Comparacion de Tecnicas de Denoising")
    print("="*65)

    psnr_noisy = psnr(orig_display, ldct_display, data_range=1.0)
    
    comparison = []
    comparison.append({
        'Tecnica': 'LDCT (con ruido)',
        'PSNR (dB)': psnr_noisy,
        'SSIM': ssim(orig_display, ldct_display, data_range=1.0)
    })

    for tech, denoised in denoised_results.items():
        comparison.append({
            'Tecnica': tech,
            'PSNR (dB)': psnr(orig_display, denoised, data_range=1.0),
            'SSIM': ssim(orig_display, denoised, data_range=1.0)
        })

    comparison_df = pd.DataFrame(comparison)
    comparison_df['PSNR (dB)'] = comparison_df['PSNR (dB)'].round(2)
    comparison_df['SSIM'] = comparison_df['SSIM'].round(4)

    # Ordenar por PSNR
    comparison_df = comparison_df.sort_values('PSNR (dB)', ascending=False)
    print(comparison_df.to_string(index=False))

    print("\n" + "="*65)
    print("Mayor PSNR y SSIM = Mejor calidad de imagen")
else:
    comparison_df = None
    print("[SKIP] Tabla comparativa - usando datos reales Mayo")

In [ ]:
# Dataset para entrenamiento con ruido simulado en dominio del sinograma
if USE_SIMULATED_NOISE:
    class CTDenoisingDataset(Dataset):
        """
        Dataset para entrenamiento de denoising en CT.
        
        Genera pares (LDCT, NDCT) simulando ruido en el dominio
        del sinograma segun Yu et al. 2012.
        """
        def __init__(self, data_path, annotations_path, patch_size=64, 
                     num_patches_per_scan=50, dose_ratio=0.25, I0=1e5):
            self.patch_size = patch_size
            self.num_patches = num_patches_per_scan
            self.dose_ratio = dose_ratio
            self.I0 = I0
            
            # Cargar datos
            self.loader = LUNA16DataLoader(data_path, annotations_path)
            self.mhd_files = list(Path(data_path).glob("*.mhd"))
            
            # Cache de slices (pares NDCT, LDCT)
            self._pairs_cache = []
            self._load_and_simulate()
        
        def _load_and_simulate(self):
            """Carga slices y simula LDCT en dominio del sinograma"""
            print("Cargando slices y simulando LDCT (Yu et al. 2012)...")
            print("  Proceso: Imagen -> Sinograma -> +Ruido -> Reconstruccion")
            
            max_scans = min(5, len(self.mhd_files))
            
            for mhd_file in tqdm(self.mhd_files[:max_scans], desc="Procesando scans"):
                ct_scan, _, _ = self.loader.load_itk_image(str(mhd_file))
                
                # Tomar slices centrales
                start_slice = ct_scan.shape[0] // 4
                end_slice = 3 * ct_scan.shape[0] // 4
                
                for slice_idx in range(start_slice, end_slice, 10):  # Cada 10 slices
                    ndct_slice = ct_scan[slice_idx]
                    
                    # Simular LDCT en dominio del sinograma
                    ldct_slice = add_ldct_noise_yu2012(
                        ndct_slice, 
                        dose_ratio=self.dose_ratio,
                        I0=self.I0
                    )
                    
                    # Normalizar ambas
                    ndct_norm = self.loader.normalize_hu(ndct_slice, -1000, 400)
                    ldct_norm = self.loader.normalize_hu(ldct_slice, -1000, 400)
                    
                    self._pairs_cache.append((ndct_norm, ldct_norm))
            
            print(f"[OK] Pares NDCT/LDCT generados: {len(self._pairs_cache)}")
        
        def __len__(self):
            return len(self._pairs_cache) * self.num_patches
        
        def _extract_random_patch(self, ndct, ldct):
            """Extrae patch aleatorio del mismo lugar en ambas imagenes"""
            h, w = ndct.shape
            
            max_y = h - self.patch_size
            max_x = w - self.patch_size
            
            if max_y <= 0 or max_x <= 0:
                return ndct[:self.patch_size, :self.patch_size], \
                       ldct[:self.patch_size, :self.patch_size]
            
            y = np.random.randint(0, max_y)
            x = np.random.randint(0, max_x)
            
            return ndct[y:y+self.patch_size, x:x+self.patch_size], \
                   ldct[y:y+self.patch_size, x:x+self.patch_size]
        
        def __getitem__(self, idx):
            # Seleccionar par
            pair_idx = idx // self.num_patches
            ndct_slice, ldct_slice = self._pairs_cache[pair_idx]
            
            # Extraer patch aleatorio
            ndct_patch, ldct_patch = self._extract_random_patch(ndct_slice, ldct_slice)
            
            # Clip para asegurar rango valido
            ndct_patch = np.clip(ndct_patch, 0, 1)
            ldct_patch = np.clip(ldct_patch, 0, 1)
            
            # Convertir a tensores
            ndct_tensor = torch.FloatTensor(ndct_patch).unsqueeze(0)
            ldct_tensor = torch.FloatTensor(ldct_patch).unsqueeze(0)
            
            # Retornar (ruidosa, limpia)
            return ldct_tensor, ndct_tensor

    print("[OK] Dataset con ruido en sinograma definido (Yu et al. 2012)")
else:
    print("[INFO] Usando datos reales Mayo - dataset simulado no necesario")

In [ ]:
# Crear dataset y dataloader (solo ruido simulado)
if USE_SIMULATED_NOISE:
    denoise_dataset = CTDenoisingDataset(
        DATA_PATH, 
        ANNOTATIONS_PATH,
        patch_size=64,
        num_patches_per_scan=100,
        dose_ratio=0.25
    )

    denoise_loader = DataLoader(
        denoise_dataset, 
        batch_size=16, 
        shuffle=True,
        num_workers=0
    )

    print(f"\nDataset size: {len(denoise_dataset)}")
    print(f"Batches: {len(denoise_loader)}")
else:
    denoise_dataset = None
    denoise_loader = None
    print("[INFO] Dataset simulado no creado - se usara Mayo")

In [ ]:
# Visualizar ejemplos del dataset (solo ruido simulado)
if USE_SIMULATED_NOISE and denoise_loader is not None:
    noisy_batch, clean_batch = next(iter(denoise_loader))

    fig, axes = plt.subplots(2, 4, figsize=(16, 8))

    for i in range(4):
        axes[0, i].imshow(noisy_batch[i, 0].numpy(), cmap='bone')
        axes[0, i].set_title('Ruidosa (LDCT)', fontsize=11)
        axes[0, i].axis('off')
        
        axes[1, i].imshow(clean_batch[i, 0].numpy(), cmap='bone')
        axes[1, i].set_title('Limpia (NDCT)', fontsize=11)
        axes[1, i].axis('off')

    plt.suptitle('Ejemplos del Dataset de Denoising (Pares Ruidosa/Limpia)', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("[SKIP] Visualizacion de dataset simulado - se usara Mayo")

In [ ]:
# Entrenar el modelo (solo ruido simulado)
if USE_SIMULATED_NOISE and denoise_loader is not None:
    print("Iniciando entrenamiento con ruido simulado...")
    print("="*60)

    history = train_denoiser(model_dncnn, denoise_loader, epochs=5, lr=1e-3)

    print("\n" + "="*60)
    print("[OK] Entrenamiento completado!")
else:
    history = None
    print("[INFO] Entrenamiento con ruido simulado omitido")
    print("       Se entrenara con datos reales Mayo en celdas posteriores")

In [ ]:
# Visualizar historial de entrenamiento (solo ruido simulado)
if history is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Loss
    axes[0].plot(history['loss'], 'b-o', linewidth=2, markersize=8)
    axes[0].set_xlabel('Epoca', fontsize=12)
    axes[0].set_ylabel('Loss (MSE)', fontsize=12)
    axes[0].set_title('Perdida durante Entrenamiento', fontsize=14)
    axes[0].grid(True, alpha=0.3)

    # PSNR
    axes[1].plot(history['psnr'], 'g-o', linewidth=2, markersize=8)
    axes[1].set_xlabel('Epoca', fontsize=12)
    axes[1].set_ylabel('PSNR (dB)', fontsize=12)
    axes[1].set_title('PSNR durante Entrenamiento', fontsize=14)
    axes[1].grid(True, alpha=0.3)

    plt.suptitle('Historial de Entrenamiento DnCNN', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("[SKIP] Historial de entrenamiento - se mostrara despues de entrenar con Mayo")

In [ ]:
# Evaluar en imagen de prueba (solo ruido simulado)
if USE_SIMULATED_NOISE and original_slice is not None:
    results = evaluate_denoiser(model_dncnn, orig_display, dose_ratio=0.25)

    print("Metricas de Evaluacion:")
    print("="*50)
    print(f"\nImagen LDCT (con ruido):")
    print(f"  PSNR: {results['psnr_noisy']:.2f} dB")
    print(f"  SSIM: {results['ssim_noisy']:.4f}")

    print(f"\nImagen Denoised (DnCNN):")
    print(f"  PSNR: {results['psnr_denoised']:.2f} dB (+{results['psnr_gain']:.2f} dB)")
    print(f"  SSIM: {results['ssim_denoised']:.4f} (+{results['ssim_gain']:.4f})")
else:
    results = None
    print("[INFO] Evaluacion con ruido simulado omitida")
    print("       Se evaluara despues de entrenar con datos Mayo")

In [ ]:
# Visualizar resultados de denoising con DnCNN (solo ruido simulado)
if results is not None:
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))

    # Fila 1: Imagenes completas
    axes[0, 0].imshow(results['clean'], cmap='bone')
    axes[0, 0].set_title('Original (NDCT)', fontsize=12)
    axes[0, 0].axis('off')

    axes[0, 1].imshow(results['noisy'], cmap='bone')
    axes[0, 1].set_title(f'LDCT Simulado\nPSNR: {results["psnr_noisy"]:.1f} dB', fontsize=12)
    axes[0, 1].axis('off')

    axes[0, 2].imshow(results['denoised'], cmap='bone')
    axes[0, 2].set_title(f'DnCNN Denoised\nPSNR: {results["psnr_denoised"]:.1f} dB', fontsize=12)
    axes[0, 2].axis('off')

    # Diferencia (residuo aprendido)
    residual = np.abs(results['noisy'] - results['denoised'])
    axes[0, 3].imshow(residual, cmap='hot')
    axes[0, 3].set_title('Ruido Removido\n(Residuo)', fontsize=12)
    axes[0, 3].axis('off')

    # Fila 2: Zoom
    y1, y2, x1, x2 = 200, 300, 200, 300

    axes[1, 0].imshow(results['clean'][y1:y2, x1:x2], cmap='bone')
    axes[1, 0].set_title('Zoom Original', fontsize=12)
    axes[1, 0].axis('off')

    axes[1, 1].imshow(results['noisy'][y1:y2, x1:x2], cmap='bone')
    axes[1, 1].set_title('Zoom LDCT', fontsize=12)
    axes[1, 1].axis('off')

    axes[1, 2].imshow(results['denoised'][y1:y2, x1:x2], cmap='bone')
    axes[1, 2].set_title('Zoom DnCNN', fontsize=12)
    axes[1, 2].axis('off')

    axes[1, 3].imshow(residual[y1:y2, x1:x2], cmap='hot')
    axes[1, 3].set_title('Zoom Residuo', fontsize=12)
    axes[1, 3].axis('off')

    plt.suptitle('Resultados de Denoising con DnCNN', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("[SKIP] Visualizacion de resultados - se mostrara despues de entrenar con Mayo")

---

## 8. Comparación: Clásico vs Deep Learning

In [ ]:
# Comparacion final de todas las tecnicas (solo ruido simulado)
if results is not None and denoised_results is not None:
    print("Comparacion Final: Clasico vs Deep Learning")
    print("="*65)

    # Aplicar denoising clasico a la misma imagen ruidosa
    classical_on_same = apply_classical_denoising(results['noisy'])

    final_comparison = []

    # LDCT (referencia)
    final_comparison.append({
        'Tecnica': 'LDCT (sin procesar)',
        'Tipo': 'Referencia',
        'PSNR (dB)': results['psnr_noisy'],
        'SSIM': results['ssim_noisy']
    })

    # Tecnicas clasicas
    for tech, denoised in classical_on_same.items():
        final_comparison.append({
            'Tecnica': tech,
            'Tipo': 'Clasico',
            'PSNR (dB)': psnr(results['clean'], denoised, data_range=1.0),
            'SSIM': ssim(results['clean'], denoised, data_range=1.0)
        })

    # DnCNN
    final_comparison.append({
        'Tecnica': 'DnCNN',
        'Tipo': 'Deep Learning',
        'PSNR (dB)': results['psnr_denoised'],
        'SSIM': results['ssim_denoised']
    })

    # Crear DataFrame y ordenar
    final_df = pd.DataFrame(final_comparison)
    final_df['PSNR (dB)'] = final_df['PSNR (dB)'].round(2)
    final_df['SSIM'] = final_df['SSIM'].round(4)
    final_df = final_df.sort_values('PSNR (dB)', ascending=False)

    print(final_df.to_string(index=False))
    print("\n" + "="*65)
else:
    final_df = None
    print("[SKIP] Comparacion final - se mostrara despues de entrenar con Mayo")

In [ ]:
# Grafico de barras comparativo (solo ruido simulado)
if final_df is not None and results is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Ordenar por PSNR para visualizacion
    df_sorted = final_df.sort_values('PSNR (dB)', ascending=True)

    # Colores por tipo
    colors = ['gray' if t == 'Referencia' else 'steelblue' if t == 'Clasico' else 'coral' 
              for t in df_sorted['Tipo']]

    # PSNR
    axes[0].barh(df_sorted['Tecnica'], df_sorted['PSNR (dB)'], color=colors)
    axes[0].set_xlabel('PSNR (dB)', fontsize=12)
    axes[0].set_title('Comparacion de PSNR', fontsize=14)
    axes[0].axvline(x=results['psnr_noisy'], color='red', linestyle='--', alpha=0.5, label='LDCT')
    axes[0].grid(True, alpha=0.3, axis='x')

    # SSIM
    axes[1].barh(df_sorted['Tecnica'], df_sorted['SSIM'], color=colors)
    axes[1].set_xlabel('SSIM', fontsize=12)
    axes[1].set_title('Comparacion de SSIM', fontsize=14)
    axes[1].axvline(x=results['ssim_noisy'], color='red', linestyle='--', alpha=0.5, label='LDCT')
    axes[1].grid(True, alpha=0.3, axis='x')

    # Leyenda
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='gray', label='Referencia'),
        Patch(facecolor='steelblue', label='Clasico'),
        Patch(facecolor='coral', label='Deep Learning')
    ]
    fig.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.98, 0.98))

    plt.suptitle('Comparacion de Tecnicas de Denoising', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("[SKIP] Grafico comparativo - se mostrara despues de entrenar con Mayo")

---

## 9. Guardar Modelo Entrenado

In [ ]:
# Guardar pesos del modelo
# weights_dir ya esta definido en celda inicial (Drive en Colab, local en PC)

model_path = os.path.join(weights_dir, 'dncnn_denoising.pth')

if USE_SIMULATED_NOISE:
    torch.save(model_dncnn.state_dict(), model_path)
    print(f"[OK] Modelo guardado en: {model_path}")
    print(f"    Tamano: {os.path.getsize(model_path) / 1024 / 1024:.2f} MB")
else:
    print("[INFO] Modelo se guardara despues de entrenar con datos Mayo")

In [ ]:
# Funcion para cargar modelo guardado
def load_denoiser(weights_path, device='cuda'):
    """
    Carga un modelo DnCNN entrenado
    
    Args:
        weights_path: Ruta al archivo .pth
        device: Dispositivo ('cuda' o 'cpu')
    
    Returns:
        model: Modelo cargado listo para inferencia
    """
    model = DnCNN(in_channels=1, out_channels=1, num_features=64, depth=17)
    model.load_state_dict(torch.load(weights_path, map_location=device))
    model.to(device)
    model.eval()
    return model


# Verificar que se puede cargar (solo si se entreno con ruido simulado)
if USE_SIMULATED_NOISE and os.path.exists(model_path):
    model_loaded = load_denoiser(model_path, device)
    print("[OK] Modelo cargado correctamente!")
else:
    model_loaded = None
    print("[INFO] Modelo se cargara despues de entrenar con Mayo")

---

# PARTE B: Datos Reales Mayo Clinic (Google Colab)

> **NOTA**: Esta seccion usa datos reales del dataset Mayo Clinic LDCT.
> Se divide en dos subsecciones para entrenar modelos especializados:
>
> - **B.1**: Proyecciones/Sinogramas (~50GB) - Para denoising en dominio de proyecciones
> - **B.2**: Imagenes CT Reconstruidas (~5GB) - Para denoising en dominio de imagen
>
> Si solo quieres trabajar con ruido simulado, puedes **colapsar esta seccion**.

---

---

## B.1: Proyecciones/Sinogramas (Mayo_LDCT)

El dataset LDCT-and-Projection-data contiene **proyecciones** (sinogramas) con diferentes niveles de dosis.

### Caracteristicas de las proyecciones:

| Aspecto | Detalle |
|---------|---------|
| **Formato** | DICOM-CT-PD (Projection Data) |
| **Tamano** | 736 x 64 por proyeccion |
| **Dosis** | Full-dose (100%) + Low-dose (25%) |
| **Tamano Total** | ~50 GB |

### Estructura esperada:
```
Mayo_LDCT/
    <SeriesInstanceUID>/    # Carpeta por serie
        *.dcm               # Proyecciones DICOM
    ...
```

> El SeriesDescription del DICOM indica "Full dose projections" o "Low dose projections"

---

In [ ]:
# ============================================================
# B.1 - PREPARAR PROYECCIONES MAYO - COPIAR A LOCAL
# ============================================================
import shutil

if MAYO_PROJ_AVAILABLE:
    # Copiar de Drive a local (mucho mas rapido para entrenamiento)
    local_proj_path = "/content/Mayo_LDCT_local"

    if IN_COLAB and not os.path.exists(local_proj_path):
        print("Copiando proyecciones de Drive a almacenamiento local...")
        print("(Esto mejora velocidad de entrenamiento significativamente)")
        shutil.copytree(MAYO_PROJ_PATH, local_proj_path)
        print(f"[OK] Datos copiados a {local_proj_path}")
    else:
        if IN_COLAB:
            print(f"[OK] Datos ya existen en {local_proj_path}")
        else:
            local_proj_path = MAYO_PROJ_PATH
            print(f"[OK] Usando datos locales: {local_proj_path}")

    # Usar path local para el resto de la seccion
    MAYO_PROJ_PATH_FAST = local_proj_path
else:
    MAYO_PROJ_PATH_FAST = None
    print("[INFO] Proyecciones Mayo no disponibles, saltando seccion B.1")

In [ ]:
# ============================================================
# LOADER PARA MAYO CLINIC - ADAPTADO A ESTRUCTURA TCIA
# ============================================================
import pydicom
from glob import glob
from collections import defaultdict


# ============================================================
# B.1 - LOADER PARA PROYECCIONES MAYO
# ============================================================
if not MAYO_PROJ_AVAILABLE:
    print("[INFO] Seccion B.1 saltada - proyecciones no disponibles")
else:
    class MayoLDCTLoader:
        """
        Cargador para dataset Mayo Clinic LDCT (TCIA)
        Adaptado a la estructura real de descarga de TCIA.

        Estructura TCIA:
        - Cada serie es una carpeta con SeriesInstanceUID
        - SeriesDescription indica "Full dose projections" o "Low dose projections"
        - Cada DICOM es una proyeccion individual (736, 64)
        """

        def __init__(self, base_path):
            self.base_path = Path(base_path)
            self.series_info = self._scan_series()
            self.patients = self._group_by_patient()

        def _scan_series(self):
            """Escanea todas las series y extrae metadata"""
            series = []

            for series_dir in self.base_path.iterdir():
                if not series_dir.is_dir():
                    continue

                dcm_files = list(series_dir.glob("*.dcm"))
                if not dcm_files:
                    continue

                # Leer metadata del primer DICOM
                try:
                    ds = pydicom.dcmread(str(dcm_files[0]), stop_before_pixels=True)
                    series.append({
                        'path': series_dir,
                        'series_uid': series_dir.name,
                        'patient_id': getattr(ds, 'PatientID', 'Unknown'),
                        'series_desc': getattr(ds, 'SeriesDescription', ''),
                        'n_slices': len(dcm_files)
                    })
                except Exception as e:
                    print(f"[WARNING] Error leyendo {series_dir.name}: {e}")

            print(f"[OK] {len(series)} series encontradas")
            return series

        def _group_by_patient(self):
            """Agrupa series por paciente, identificando pares full/low dose"""
            patients = defaultdict(dict)

            for s in self.series_info:
                pid = s['patient_id']
                desc = s['series_desc'].lower()

                if 'full' in desc:
                    patients[pid]['full'] = s
                elif 'low' in desc:
                    patients[pid]['low'] = s

            # Filtrar solo pacientes con ambas series
            valid_patients = {
                pid: data for pid, data in patients.items()
                if 'full' in data and 'low' in data
            }

            print(f"[OK] {len(valid_patients)} pacientes con pares completos")
            for pid, data in valid_patients.items():
                print(f"    {pid}: Full={data['full']['n_slices']}, Low={data['low']['n_slices']} projections")

            return valid_patients

        def load_patient_pair(self, patient_id, slice_idx=None):
            """
            Carga un par de sinogramas (full/low dose) para un paciente

            Args:
                patient_id: ID del paciente
                slice_idx: Indice del slice/proyeccion (None = retorna paths)

            Returns:
                Si slice_idx es None: (full_files, low_files)
                Si slice_idx es int: (full_array, low_array)
            """
            if patient_id not in self.patients:
                raise ValueError(f"Paciente {patient_id} no encontrado o sin par completo")

            full_path = self.patients[patient_id]['full']['path']
            low_path = self.patients[patient_id]['low']['path']

            full_files = sorted(full_path.glob("*.dcm"))
            low_files = sorted(low_path.glob("*.dcm"))

            if slice_idx is None:
                return full_files, low_files

            # Cargar slice especifico
            full_ds = pydicom.dcmread(str(full_files[slice_idx]))
            low_ds = pydicom.dcmread(str(low_files[slice_idx]))

            return full_ds.pixel_array, low_ds.pixel_array


    # Crear loader
    if MAYO_AVAILABLE:
        mayo_loader = MayoLDCTLoader(MAYO_PATH_FAST)
        print(f"\nPacientes disponibles: {list(mayo_loader.patients.keys())}")
    else:
        mayo_loader = None
        print("[INFO] Mayo loader no creado - usando ruido simulado")

In [ ]:
# ============================================================
# B.1 - DATASET DE SINOGRAMAS MAYO
# ============================================================
from skimage.transform import resize

if not MAYO_PROJ_AVAILABLE:
    train_loader_mayo = None
    test_loader_mayo = None
    print("[INFO] Dataset sinogramas saltado - proyecciones no disponibles")
else:
    class MayoSinogramDataset(Dataset):
        """
        Dataset de pares de sinogramas Full/Low dose.
        Usa sinogramas completos (no patches) con resize a tamano fijo.
        """

        def __init__(self, loader, patient_ids=None, samples_per_patient=200,
                     target_size=(64, 64), augment=False):
            """
            Args:
                loader: MayoLDCTLoader
                patient_ids: Lista de pacientes (None = todos)
                samples_per_patient: Proyecciones a muestrear por paciente
                target_size: Tamano de salida (resize)
                augment: Aplicar data augmentation
            """
            self.loader = loader
            self.target_size = target_size
            self.augment = augment

            if patient_ids is None:
                self.patient_ids = list(loader.patients.keys())
            else:
                self.patient_ids = [p for p in patient_ids if p in loader.patients]

            # Crear lista de samples (pid, slice_idx)
            self.samples = []
            for pid in self.patient_ids:
                n_slices = loader.patients[pid]['full']['n_slices']
                # Muestrear uniformemente
                indices = np.linspace(0, n_slices-1, min(samples_per_patient, n_slices), dtype=int)
                for idx in indices:
                    self.samples.append((pid, int(idx)))

            print(f"[OK] Dataset: {len(self.samples)} sinogramas de {len(self.patient_ids)} pacientes")

        def __len__(self):
            return len(self.samples)

        def __getitem__(self, idx):
            pid, slice_idx = self.samples[idx]

            # Cargar sinogramas
            full_sino, low_sino = self.loader.load_patient_pair(pid, slice_idx)

            # Convertir a float32
            full_sino = full_sino.astype(np.float32)
            low_sino = low_sino.astype(np.float32)

            # Normalizar a [0, 1]
            def normalize(x):
                x_min, x_max = x.min(), x.max()
                if x_max - x_min > 0:
                    return (x - x_min) / (x_max - x_min)
                return x

            full_sino = normalize(full_sino)
            low_sino = normalize(low_sino)

            # Resize a tamano objetivo (usando sinograma completo)
            if full_sino.shape != self.target_size:
                full_sino = resize(full_sino, self.target_size, preserve_range=True, anti_aliasing=True)
                low_sino = resize(low_sino, self.target_size, preserve_range=True, anti_aliasing=True)

            # Data augmentation
            if self.augment:
                if np.random.rand() > 0.5:
                    full_sino = np.fliplr(full_sino).copy()
                    low_sino = np.fliplr(low_sino).copy()

            # Convertir a tensor (C, H, W)
            full_tensor = torch.from_numpy(full_sino).unsqueeze(0).float()
            low_tensor = torch.from_numpy(low_sino).unsqueeze(0).float()

            return low_tensor, full_tensor  # (noisy, clean)

    # Instanciar loader con path local (rapido)
    loader_proj = MayoLDCTLoader(MAYO_PROJ_PATH_FAST)

    # Dividir pacientes en train/test
    patient_ids = loader_proj.get_patient_ids()
    n_train = max(1, int(len(patient_ids) * 0.8))
    train_patients = patient_ids[:n_train]
    test_patients = patient_ids[n_train:] if n_train < len(patient_ids) else patient_ids[:1]

    print(f"\nDivision de datos proyecciones:")
    print(f"  Train: {train_patients}")
    print(f"  Test: {test_patients}")

    # Crear datasets
    train_dataset_mayo = MayoSinogramDataset(
        loader_proj,
        patient_ids=train_patients,
        samples_per_patient=200,
        target_size=(64, 64),
        augment=True
    )

    test_dataset_mayo = MayoSinogramDataset(
        loader_proj,
        patient_ids=test_patients,
        samples_per_patient=50,
        target_size=(64, 64),
        augment=False
    )

    # Crear dataloaders
    train_loader_mayo = DataLoader(train_dataset_mayo, batch_size=16, shuffle=True, num_workers=0)
    test_loader_mayo = DataLoader(test_dataset_mayo, batch_size=16, shuffle=False, num_workers=0)

    print(f"\n[OK] DataLoaders creados:")
    print(f"  Train: {len(train_loader_mayo)} batches")
    print(f"  Test: {len(test_loader_mayo)} batches")

In [ ]:
# Visualizar ejemplos de sinogramas
if train_loader_mayo is not None:
    low_batch, full_batch = next(iter(train_loader_mayo))

    fig, axes = plt.subplots(2, 4, figsize=(16, 8))

    for i in range(min(4, low_batch.size(0))):
        axes[0, i].imshow(low_batch[i, 0].numpy(), cmap='gray', aspect='auto')
        axes[0, i].set_title(f'Low-dose sinogram', fontsize=11)
        axes[0, i].axis('off')

        axes[1, i].imshow(full_batch[i, 0].numpy(), cmap='gray', aspect='auto')
        axes[1, i].set_title(f'Full-dose sinogram', fontsize=11)
        axes[1, i].axis('off')

    plt.suptitle('Pares de Sinogramas Mayo Clinic (64x64)', fontsize=14)
    plt.tight_layout()
    plt.show()

    # Mostrar diferencia
    diff = torch.abs(full_batch - low_batch)
    print(f"\nEstadisticas:")
    print(f"  Diferencia media: {diff.mean():.4f}")
    print(f"  Diferencia max: {diff.max():.4f}")
else:
    print("[INFO] No hay datos de proyecciones Mayo para visualizar")

---

## 11. Entrenamiento con Datos Reales Mayo Clinic

Si el dataset Mayo está disponible, podemos entrenar el modelo DnCNN con pares reales NDCT/LDCT.

In [ ]:
# ============================================================
# B.1 - ENTRENAMIENTO CON CHECKPOINTS - SINOGRAMAS MAYO
# ============================================================

def train_denoiser_with_checkpoints(model, train_loader, test_loader=None,
                                     epochs=10, lr=1e-3, save_dir=None):
    """
    Entrena el modelo con guardado de checkpoints cada epoca.
    """
    model.train()

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

    history = {'loss': [], 'psnr': [], 'val_psnr': []}
    best_psnr = 0

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        epoch_psnr = 0
        num_batches = 0

        pbar = tqdm(train_loader, desc=f'Epoca {epoch+1}/{epochs}')

        for noisy, clean in pbar:
            noisy = noisy.to(device)
            clean = clean.to(device)

            optimizer.zero_grad()
            denoised = model(noisy)
            loss = criterion(denoised, clean)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            with torch.no_grad():
                mse = F.mse_loss(denoised, clean)
                batch_psnr = 10 * torch.log10(1.0 / (mse + 1e-10))
                epoch_psnr += batch_psnr.item()

            num_batches += 1
            pbar.set_postfix({'Loss': f'{loss.item():.6f}', 'PSNR': f'{batch_psnr.item():.2f}'})

        scheduler.step()

        avg_loss = epoch_loss / num_batches
        avg_psnr = epoch_psnr / num_batches

        history['loss'].append(avg_loss)
        history['psnr'].append(avg_psnr)

        # Validacion en test set
        val_psnr = 0
        if test_loader is not None:
            model.eval()
            val_psnr_list = []
            with torch.no_grad():
                for noisy, clean in test_loader:
                    noisy = noisy.to(device)
                    clean = clean.to(device)
                    denoised = model(noisy)
                    mse = F.mse_loss(denoised, clean)
                    val_psnr_list.append(10 * torch.log10(1.0 / (mse + 1e-10)).item())
            val_psnr = np.mean(val_psnr_list)
            history['val_psnr'].append(val_psnr)
            model.train()

        print(f"Epoca {epoch+1}: Loss={avg_loss:.6f}, PSNR={avg_psnr:.2f}dB, Val_PSNR={val_psnr:.2f}dB")

        # Guardar checkpoint
        if save_dir:
            checkpoint_path = os.path.join(save_dir, f'checkpoint_epoch_{epoch+1}.pth')
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss,
                'psnr': avg_psnr,
            }, checkpoint_path)

            # Guardar mejor modelo
            if avg_psnr > best_psnr:
                best_psnr = avg_psnr
                best_path = os.path.join(save_dir, 'best_model.pth')
                torch.save(model.state_dict(), best_path)

    return history, model


# Entrenar modelo para sinogramas
if train_loader_mayo is not None:
    print("="*60)
    print("ENTRENAMIENTO - MODELO DNCNN PARA SINOGRAMAS")
    print("="*60)

    # Crear modelo
    model_mayo = DnCNN(channels=1, num_layers=17, features=64).to(device)
    print(f"[OK] Modelo DnCNN creado en {device}")

    # Directorio para checkpoints
    sino_save_dir = os.path.join(weights_dir, "dncnn_sinograms_checkpoints")
    os.makedirs(sino_save_dir, exist_ok=True)

    # Entrenar
    history_mayo, model_mayo = train_denoiser_with_checkpoints(
        model_mayo,
        train_loader_mayo,
        test_loader=test_loader_mayo,
        epochs=15,
        lr=1e-3,
        save_dir=sino_save_dir
    )

    # Guardar modelo final
    final_path = os.path.join(weights_dir, "dncnn_sinograms_final.pth")
    torch.save(model_mayo.state_dict(), final_path)
    print(f"\n[OK] Modelo sinogramas guardado: {final_path}")
else:
    model_mayo = None
    history_mayo = None
    print("[INFO] Entrenamiento sinogramas saltado - datos no disponibles")

In [ ]:
# ============================================================
# B.1 - EVALUACION FINAL EN TEST SET - SINOGRAMAS
# ============================================================
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

if model_mayo is not None and test_loader_mayo is not None:
    print("="*60)
    print("EVALUACION EN TEST SET - MODELO SINOGRAMAS")
    print("="*60)

    model_mayo.eval()

    test_psnr_ldct = []
    test_psnr_restored = []
    test_ssim_ldct = []
    test_ssim_restored = []

    examples_sino = {'ldct': [], 'ndct': [], 'restored': []}

    with torch.no_grad():
        for ldct, ndct in tqdm(test_loader_mayo, desc="Evaluando test set sinogramas"):
            ldct = ldct.to(device)
            ndct_gpu = ndct.to(device)
            restored = model_mayo(ldct)

            for i in range(ldct.size(0)):
                ldct_np = ldct[i, 0].cpu().numpy()
                ndct_np = ndct[i, 0].cpu().numpy()
                restored_np = restored[i, 0].cpu().numpy()

                test_psnr_ldct.append(psnr(ndct_np, ldct_np, data_range=1.0))
                test_ssim_ldct.append(ssim(ndct_np, ldct_np, data_range=1.0))

                test_psnr_restored.append(psnr(ndct_np, restored_np, data_range=1.0))
                test_ssim_restored.append(ssim(ndct_np, restored_np, data_range=1.0))

                if len(examples_sino['ldct']) < 8:
                    examples_sino['ldct'].append(ldct_np)
                    examples_sino['ndct'].append(ndct_np)
                    examples_sino['restored'].append(restored_np)

    # Resultados
    print("\n" + "-"*60)
    print("METRICAS EN TEST SET - MODELO SINOGRAMAS")
    print("-"*60)
    print(f"{'Metrica':<20} {'LDCT (ruidosa)':<20} {'Restaurada':<20} {'Ganancia':<15}")
    print("-"*60)

    psnr_ldct_mean = np.mean(test_psnr_ldct)
    psnr_restored_mean = np.mean(test_psnr_restored)
    ssim_ldct_mean = np.mean(test_ssim_ldct)
    ssim_restored_mean = np.mean(test_ssim_restored)

    print(f"{'PSNR (dB)':<20} {psnr_ldct_mean:<20.2f} {psnr_restored_mean:<20.2f} {'+' if psnr_restored_mean > psnr_ldct_mean else ''}{psnr_restored_mean - psnr_ldct_mean:.2f}")
    print(f"{'SSIM':<20} {ssim_ldct_mean:<20.4f} {ssim_restored_mean:<20.4f} {'+' if ssim_restored_mean > ssim_ldct_mean else ''}{ssim_restored_mean - ssim_ldct_mean:.4f}")

    # Visualizar ejemplos
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))

    for i in range(min(4, len(examples_sino['ldct']))):
        axes[0, i].imshow(examples_sino['ldct'][i], cmap='gray', aspect='auto')
        axes[0, i].set_title(f'Low-dose sinogram')
        axes[0, i].axis('off')

        axes[1, i].imshow(examples_sino['restored'][i], cmap='gray', aspect='auto')
        axes[1, i].set_title(f'Restaurado (DnCNN)')
        axes[1, i].axis('off')

        axes[2, i].imshow(examples_sino['ndct'][i], cmap='gray', aspect='auto')
        axes[2, i].set_title(f'Full-dose (Ground Truth)')
        axes[2, i].axis('off')

    plt.suptitle('Evaluacion Modelo DnCNN-Sinogramas en Test Set', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("[INFO] Evaluacion sinogramas saltada - modelo no disponible")

---

## B.2: Imagenes CT Reconstruidas (Mayo_Images)

El dataset tambien contiene **imagenes CT reconstruidas** que son mas faciles de interpretar visualmente.

### Caracteristicas de las imagenes:

| Aspecto | Detalle |
|---------|---------|
| **Formato** | DICOM estandar (CT) |
| **Tamano** | 512 x 512 por slice |
| **Dosis** | Full Dose Images + Low Dose Images |
| **Tamano Total** | ~5 GB |

### Estructura esperada:
```
Mayo_Images/
    <SeriesInstanceUID>/    # Carpeta por serie
        *.dcm               # Slices CT DICOM
    ...
```

> El SeriesDescription del DICOM indica "Full Dose Images" o "Low Dose Images"

---

In [ ]:
# ============================================================
# B.2 - PREPARAR IMAGENES CT MAYO - COPIAR A LOCAL
# ============================================================

if MAYO_IMG_AVAILABLE:
    # Copiar de Drive a local
    local_img_path = "/content/Mayo_Images_local"

    if IN_COLAB and not os.path.exists(local_img_path):
        print("Copiando imagenes CT de Drive a almacenamiento local...")
        shutil.copytree(MAYO_IMG_PATH, local_img_path)
        print(f"[OK] Datos copiados a {local_img_path}")
    else:
        if IN_COLAB:
            print(f"[OK] Datos ya existen en {local_img_path}")
        else:
            local_img_path = MAYO_IMG_PATH
            print(f"[OK] Usando datos locales: {local_img_path}")

    MAYO_IMG_PATH_FAST = local_img_path
else:
    MAYO_IMG_PATH_FAST = None
    print("[INFO] Imagenes CT Mayo no disponibles, saltando seccion B.2")

In [ ]:
# ============================================================
# B.2 - LOADER PARA IMAGENES CT MAYO
# ============================================================
if not MAYO_IMG_AVAILABLE:
    print("[INFO] Seccion B.2 saltada - imagenes CT no disponibles")
    loader_img = None
else:
    class MayoCTImageLoader:
        """
        Cargador para imagenes CT reconstruidas Mayo Clinic (TCIA)

        Estructura TCIA:
        - Cada serie es una carpeta con SeriesInstanceUID
        - SeriesDescription indica "Full Dose Images" o "Low Dose Images"
        - Cada DICOM es un slice CT (512x512)
        """

        def __init__(self, base_path):
            self.base_path = Path(base_path)
            self.series_info = self._scan_series()
            self.patients = self._group_by_patient()

        def _scan_series(self):
            """Escanea todas las series y extrae metadata"""
            series = []

            for series_dir in self.base_path.iterdir():
                if not series_dir.is_dir():
                    continue

                dcm_files = list(series_dir.glob("*.dcm"))
                if not dcm_files:
                    continue

                # Leer metadata del primer DICOM
                try:
                    ds = pydicom.dcmread(str(dcm_files[0]), stop_before_pixels=True)
                    series.append({
                        'path': series_dir,
                        'series_uid': series_dir.name,
                        'patient_id': getattr(ds, 'PatientID', 'Unknown'),
                        'series_desc': getattr(ds, 'SeriesDescription', ''),
                        'n_slices': len(dcm_files)
                    })
                except Exception as e:
                    print(f"[WARNING] Error leyendo {series_dir.name}: {e}")

            print(f"[OK] {len(series)} series de imagenes CT encontradas")
            return series

        def _group_by_patient(self):
            """Agrupa series por paciente, identificando pares full/low dose"""
            patients = defaultdict(dict)

            for s in self.series_info:
                pid = s['patient_id']
                desc = s['series_desc'].lower()

                # Imagenes CT usan "Full Dose Images" y "Low Dose Images"
                if 'full' in desc and 'image' in desc:
                    patients[pid]['full'] = s
                elif 'low' in desc and 'image' in desc:
                    patients[pid]['low'] = s

            # Solo mantener pacientes con ambas series
            complete = {pid: data for pid, data in patients.items()
                       if 'full' in data and 'low' in data}

            print(f"[OK] {len(complete)} pacientes con pares completos de imagenes")
            return complete

        def load_patient_pair(self, patient_id, slice_idx=0):
            """Carga un par de slices CT (full/low dose)"""
            patient = self.patients.get(patient_id)
            if not patient:
                raise ValueError(f"Paciente {patient_id} no encontrado")

            full_path = patient['full']['path']
            low_path = patient['low']['path']

            # Obtener archivos ordenados
            full_files = sorted(full_path.glob("*.dcm"))
            low_files = sorted(low_path.glob("*.dcm"))

            # Ajustar indice
            slice_idx = min(slice_idx, len(full_files)-1, len(low_files)-1)

            # Cargar DICOMs
            full_ds = pydicom.dcmread(str(full_files[slice_idx]))
            low_ds = pydicom.dcmread(str(low_files[slice_idx]))

            # Extraer pixel array y aplicar rescale
            full_img = full_ds.pixel_array.astype(np.float32)
            low_img = low_ds.pixel_array.astype(np.float32)

            # Aplicar rescale slope/intercept si existen
            if hasattr(full_ds, 'RescaleSlope'):
                full_img = full_img * full_ds.RescaleSlope + full_ds.RescaleIntercept
                low_img = low_img * low_ds.RescaleSlope + low_ds.RescaleIntercept

            return full_img, low_img

        def get_patient_ids(self):
            return list(self.patients.keys())

    # Instanciar loader
    loader_img = MayoCTImageLoader(MAYO_IMG_PATH_FAST)

    # Mostrar pacientes disponibles
    print(f"\nPacientes disponibles: {loader_img.get_patient_ids()}")

In [ ]:
# ============================================================
# B.2 - DATASET DE IMAGENES CT MAYO
# ============================================================
if not MAYO_IMG_AVAILABLE:
    train_loader_img = None
    test_loader_img = None
else:
    class MayoCTDataset(Dataset):
        """
        Dataset de pares de imagenes CT Full/Low dose.
        """

        def __init__(self, loader, patient_ids=None, slices_per_patient=50,
                     target_size=(128, 128), augment=False):
            """
            Args:
                loader: MayoCTImageLoader
                patient_ids: Lista de pacientes (None = todos)
                slices_per_patient: Slices a muestrear por paciente
                target_size: Tamano de salida (resize)
                augment: Aplicar data augmentation
            """
            self.loader = loader
            self.target_size = target_size
            self.augment = augment

            if patient_ids is None:
                self.patient_ids = list(loader.patients.keys())
            else:
                self.patient_ids = [p for p in patient_ids if p in loader.patients]

            # Crear lista de samples (pid, slice_idx)
            self.samples = []
            for pid in self.patient_ids:
                n_slices = loader.patients[pid]['full']['n_slices']
                # Muestrear uniformemente
                indices = np.linspace(0, n_slices-1, min(slices_per_patient, n_slices), dtype=int)
                for idx in indices:
                    self.samples.append((pid, int(idx)))

            print(f"[OK] Dataset CT: {len(self.samples)} slices de {len(self.patient_ids)} pacientes")

        def __len__(self):
            return len(self.samples)

        def __getitem__(self, idx):
            pid, slice_idx = self.samples[idx]

            # Cargar imagenes CT
            full_img, low_img = self.loader.load_patient_pair(pid, slice_idx)

            # Convertir a float32
            full_img = full_img.astype(np.float32)
            low_img = low_img.astype(np.float32)

            # Normalizar HU a [0, 1] (ventana -1000 a 400 HU)
            def normalize_hu(img, window_min=-1000, window_max=400):
                img = np.clip(img, window_min, window_max)
                return (img - window_min) / (window_max - window_min)

            full_img = normalize_hu(full_img)
            low_img = normalize_hu(low_img)

            # Resize si es necesario
            if full_img.shape != self.target_size:
                full_img = resize(full_img, self.target_size, preserve_range=True, anti_aliasing=True)
                low_img = resize(low_img, self.target_size, preserve_range=True, anti_aliasing=True)

            # Data augmentation
            if self.augment:
                if np.random.rand() > 0.5:
                    full_img = np.fliplr(full_img).copy()
                    low_img = np.fliplr(low_img).copy()
                if np.random.rand() > 0.5:
                    full_img = np.flipud(full_img).copy()
                    low_img = np.flipud(low_img).copy()

            # Convertir a tensor (C, H, W)
            full_tensor = torch.from_numpy(full_img).unsqueeze(0).float()
            low_tensor = torch.from_numpy(low_img).unsqueeze(0).float()

            return low_tensor, full_tensor  # (noisy, clean)

    # Dividir pacientes en train/test
    patient_ids = loader_img.get_patient_ids()
    n_train = max(1, int(len(patient_ids) * 0.8))
    train_patients = patient_ids[:n_train]
    test_patients = patient_ids[n_train:] if n_train < len(patient_ids) else patient_ids[:1]

    print(f"\nDivision de datos CT:")
    print(f"  Train: {train_patients}")
    print(f"  Test: {test_patients}")

    # Crear datasets
    train_dataset_img = MayoCTDataset(
        loader_img,
        patient_ids=train_patients,
        slices_per_patient=50,
        target_size=(128, 128),
        augment=True
    )

    test_dataset_img = MayoCTDataset(
        loader_img,
        patient_ids=test_patients,
        slices_per_patient=20,
        target_size=(128, 128),
        augment=False
    )

    # Crear dataloaders
    train_loader_img = DataLoader(train_dataset_img, batch_size=8, shuffle=True, num_workers=0)
    test_loader_img = DataLoader(test_dataset_img, batch_size=8, shuffle=False, num_workers=0)

    print(f"\n[OK] DataLoaders CT creados:")
    print(f"  Train: {len(train_loader_img)} batches")
    print(f"  Test: {len(test_loader_img)} batches")

In [ ]:
# Visualizar ejemplos de imagenes CT
if train_loader_img is not None:
    low_batch, full_batch = next(iter(train_loader_img))

    fig, axes = plt.subplots(2, 4, figsize=(16, 8))

    for i in range(min(4, low_batch.size(0))):
        axes[0, i].imshow(low_batch[i, 0].numpy(), cmap='gray')
        axes[0, i].set_title(f'Low-dose CT', fontsize=11)
        axes[0, i].axis('off')

        axes[1, i].imshow(full_batch[i, 0].numpy(), cmap='gray')
        axes[1, i].set_title(f'Full-dose CT', fontsize=11)
        axes[1, i].axis('off')

    plt.suptitle('Pares de Imagenes CT Mayo Clinic (128x128)', fontsize=14)
    plt.tight_layout()
    plt.show()

    # Mostrar diferencia
    diff = torch.abs(full_batch - low_batch)
    print(f"\nEstadisticas:")
    print(f"  Diferencia media: {diff.mean():.4f}")
    print(f"  Diferencia max: {diff.max():.4f}")
else:
    print("[INFO] No hay imagenes CT Mayo para visualizar")

---

## 13. Entrenamiento con Imagenes CT Mayo

Entrenamos un segundo modelo DnCNN especializado en imagenes CT reconstruidas.

---

In [ ]:
# ============================================================
# B.2 - ENTRENAMIENTO CON IMAGENES CT
# ============================================================

if train_loader_img is not None:
    print("="*60)
    print("ENTRENAMIENTO - MODELO DNCNN PARA IMAGENES CT")
    print("="*60)

    # Crear modelo para imagenes CT
    model_ct = DnCNN(channels=1, num_layers=17, features=64).to(device)
    print(f"[OK] Modelo DnCNN-CT creado en {device}")

    # Directorio para checkpoints
    ct_save_dir = os.path.join(weights_dir, "dncnn_ct_checkpoints")
    os.makedirs(ct_save_dir, exist_ok=True)

    # Entrenar
    history_ct, model_ct = train_denoiser_with_checkpoints(
        model_ct,
        train_loader_img,
        test_loader=test_loader_img,
        epochs=15,
        lr=1e-3,
        save_dir=ct_save_dir
    )

    # Guardar modelo final
    final_path = os.path.join(weights_dir, "dncnn_ct_final.pth")
    torch.save(model_ct.state_dict(), final_path)
    print(f"\n[OK] Modelo CT guardado: {final_path}")
else:
    model_ct = None
    history_ct = None
    print("[INFO] Entrenamiento CT saltado - datos no disponibles")

In [ ]:
# ============================================================
# B.2 - EVALUACION MODELO CT EN TEST SET
# ============================================================

if model_ct is not None and test_loader_img is not None:
    print("="*60)
    print("EVALUACION EN TEST SET - MODELO CT")
    print("="*60)

    model_ct.eval()

    test_psnr_ldct = []
    test_psnr_restored = []
    test_ssim_ldct = []
    test_ssim_restored = []

    examples_ct = {'ldct': [], 'ndct': [], 'restored': []}

    with torch.no_grad():
        for ldct, ndct in tqdm(test_loader_img, desc="Evaluando test set CT"):
            ldct = ldct.to(device)
            ndct_gpu = ndct.to(device)
            restored = model_ct(ldct)

            for i in range(ldct.size(0)):
                ldct_np = ldct[i, 0].cpu().numpy()
                ndct_np = ndct[i, 0].cpu().numpy()
                restored_np = restored[i, 0].cpu().numpy()

                test_psnr_ldct.append(psnr(ndct_np, ldct_np, data_range=1.0))
                test_ssim_ldct.append(ssim(ndct_np, ldct_np, data_range=1.0))

                test_psnr_restored.append(psnr(ndct_np, restored_np, data_range=1.0))
                test_ssim_restored.append(ssim(ndct_np, restored_np, data_range=1.0))

                if len(examples_ct['ldct']) < 8:
                    examples_ct['ldct'].append(ldct_np)
                    examples_ct['ndct'].append(ndct_np)
                    examples_ct['restored'].append(restored_np)

    # Resultados
    print("\n" + "-"*60)
    print("METRICAS EN TEST SET - MODELO CT")
    print("-"*60)
    print(f"{'Metrica':<20} {'LDCT (ruidosa)':<20} {'Restaurada':<20} {'Ganancia':<15}")
    print("-"*60)

    psnr_ldct_mean = np.mean(test_psnr_ldct)
    psnr_restored_mean = np.mean(test_psnr_restored)
    ssim_ldct_mean = np.mean(test_ssim_ldct)
    ssim_restored_mean = np.mean(test_ssim_restored)

    print(f"{'PSNR (dB)':<20} {psnr_ldct_mean:<20.2f} {psnr_restored_mean:<20.2f} {'+' if psnr_restored_mean > psnr_ldct_mean else ''}{psnr_restored_mean - psnr_ldct_mean:.2f}")
    print(f"{'SSIM':<20} {ssim_ldct_mean:<20.4f} {ssim_restored_mean:<20.4f} {'+' if ssim_restored_mean > ssim_ldct_mean else ''}{ssim_restored_mean - ssim_ldct_mean:.4f}")

    # Visualizar ejemplos
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))

    for i in range(min(4, len(examples_ct['ldct']))):
        axes[0, i].imshow(examples_ct['ldct'][i], cmap='gray')
        axes[0, i].set_title(f'Low-dose CT')
        axes[0, i].axis('off')

        axes[1, i].imshow(examples_ct['restored'][i], cmap='gray')
        axes[1, i].set_title(f'Restaurada (DnCNN)')
        axes[1, i].axis('off')

        axes[2, i].imshow(examples_ct['ndct'][i], cmap='gray')
        axes[2, i].set_title(f'Full-dose (Ground Truth)')
        axes[2, i].axis('off')

    plt.suptitle('Evaluacion Modelo DnCNN-CT en Test Set', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("[INFO] Evaluacion CT saltada - modelo no disponible")

---

## 14. Comparacion de Modelos

Comparamos los dos modelos entrenados:
1. **DnCNN-Proyecciones**: Entrenado con sinogramas/proyecciones
2. **DnCNN-CT**: Entrenado con imagenes CT reconstruidas

---

In [ ]:
# ============================================================
# COMPARACION DE MODELOS ENTRENADOS
# ============================================================

print("="*60)
print("RESUMEN DE MODELOS ENTRENADOS")
print("="*60)

modelos = []

# Modelo para proyecciones
if 'model_mayo' in dir() and model_mayo is not None:
    modelos.append({
        'Nombre': 'DnCNN-Proyecciones',
        'Dominio': 'Sinogramas (736x64 -> 64x64)',
        'Datos': 'Mayo_LDCT',
        'Archivo': 'dncnn_sinograms_final.pth'
    })
    print(f"[OK] DnCNN-Proyecciones entrenado")

# Modelo para imagenes CT
if 'model_ct' in dir() and model_ct is not None:
    modelos.append({
        'Nombre': 'DnCNN-CT',
        'Dominio': 'Imagenes CT (512x512 -> 128x128)',
        'Datos': 'Mayo_Images',
        'Archivo': 'dncnn_ct_final.pth'
    })
    print(f"[OK] DnCNN-CT entrenado")

if modelos:
    print("\n" + "-"*60)
    df_modelos = pd.DataFrame(modelos)
    print(df_modelos.to_string(index=False))
    print("-"*60)

    print(f"\nModelos guardados en: {weights_dir}")
else:
    print("[INFO] No hay modelos entrenados para comparar")

---

## 15. Resumen y Referencias

### Logros de este notebook:

1. **Simulacion de ruido LDCT** a partir de datos LUNA16 usando el modelo de Yu et al. 2012
2. **Comparacion de tecnicas clasicas** de denoising (Gaussian, Bilateral, NLM, BM3D)
3. **Implementacion de DnCNN** para denoising de CT
4. **Entrenamiento con datos reales Mayo**:
   - Modelo DnCNN-Proyecciones (sinogramas)
   - Modelo DnCNN-CT (imagenes reconstruidas)

### Referencias:

- Yu et al. (2012) - "Development and validation of a practical lower-dose-simulation tool for optimizing CT scan protocols"
- Zhang et al. (2017) - "Beyond a Gaussian Denoiser: Residual Learning of Deep CNN for Image Denoising" (DnCNN)
- McCollough et al. (2017) - "Low-dose CT for the detection and classification of metastatic liver cancer" (Mayo Dataset)

---

In [ ]:
print("="*60)
print("RESUMEN DEL NOTEBOOK 05: DENOISING")
print("="*60)

print("\n1. MODELOS DE RUIDO:")
print("   - Gaussiano (aditivo)")
print("   - Poisson (cuantico)")
print("   - LDCT realista (combinado)")

print("\n2. TECNICAS DE DENOISING:")
print("   Clasicas: Gaussiano, Mediana, Bilateral, TV, NLM")
print("   Deep Learning: DnCNN (17 capas, residual learning)")

print("\n3. DATASETS SOPORTADOS:")
if MAYO_AVAILABLE:
    mayo_patients = len(mayo_loader.patients) if mayo_loader else 0
    print(f"   - Mayo Clinic: Pares reales NDCT/LDCT ({mayo_patients} pacientes)")
else:
    print("   - LUNA16: Ruido simulado")
    print("   - Mayo Clinic: No descargado (ver instrucciones arriba)")

print("\n4. RESULTADOS:")
if results is not None:
    print(f"   LDCT sin procesar: {results['psnr_noisy']:.2f} dB")
    print(f"   DnCNN:             {results['psnr_denoised']:.2f} dB (+{results['psnr_gain']:.2f} dB)")
elif MAYO_AVAILABLE:
    print("   Ver seccion de entrenamiento con datos Mayo")
else:
    print("   Ejecutar celdas de entrenamiento primero")

print("\n5. ARCHIVOS GENERADOS:")
print(f"   - {weights_dir}/dncnn_denoising.pth (ruido simulado)")
if MAYO_AVAILABLE:
    print(f"   - {weights_dir}/dncnn_mayo_real.pth (datos reales)")

print("\n" + "="*60)
print("Para mejorar resultados:")
print("  1. Descargar Mayo Clinic LDCT de TCIA (si no disponible)")
print("  2. Entrenar mas epocas (50-100)")
print("  3. Usar arquitecturas avanzadas (RED-CNN, WGAN)")
print("="*60)